In [34]:
using JSON, CSV, DataFrames; rd, wr = CSV.read, CSV.write
using GeoInterface, LibGEOS, GeoJSON
using DataStructures

In [35]:
tracts = rd("../../data/info/ctracts.csv") |> DataFrame;
minc   = rd("../../data/info/ctmedian-income.csv") |> DataFrame;

In [36]:
S = findall(n -> string(n)[1] == '5', tracts[:,:tract]); deleterows!(tracts, S);

,tract,name,travel_time,white,black
,Int64,String,Float64,Float64,Float64
1,1000100,park-cemetery-etc-Manhattan,NaN,NaN,NaN
2,1000201,Lower East Side,56.0863,27.6404,5.8427
3,1000202,Lower East Side,58.5324,36.2151,16.754
4,1000500,park-cemetery-etc-Manhattan,73.0828,NaN,NaN
5,1000600,Lower East Side,50.6527,12.5035,8.59266
6,1000700,Battery Park City-Lower Manhattan,70.732,71.8917,2.24245
7,1000800,Chinatown,52.9795,10.5925,0.967846
8,1000900,Battery Park City-Lower Manhattan,60.4484,74.5528,2.94287
9,1001001,Lower East Side,58.9823,73.468,2.89562


In [39]:
nj = OrderedDict(); nj["type"] = "FeatureCollection"; 
k = findall(n -> !occursin("park-cemetery-etc",n) && !occursin("Airport",n),tracts[:,:name])
n = names(tracts)
feats = []; url = "https://lubeso.github.io/pathways/figs"

for i ∈ k
    feat = OrderedDict("properties" => OrderedDict(),
                "geometry" => OrderedDict(),
                "type" => "Feature")
    for name ∈ n
        feat["properties"][String(name)] = tracts[i,name]
    end
    
    feat["properties"]["beeswarm"] = 
    "<img src='$url/$(tracts[i,:tract]).svg' height = '300' width = '300'>"
    
    geom = LibGEOS._readgeom(tracts[i,:geometry])
    poly = LibGEOS.MultiPolygon(geom) |> GeoInterface.MultiPolygon
    feat["geometry"]["coordinates"] = poly.coordinates
    feat["geometry"]["type"] = "MultiPolygon"
    
    push!(feats, feat)
end

nj["features"] = feats;

In [40]:
open("../../data/geo/ctracts-minus-staten.geojson", "w") do f
    write(f, JSON.json(nj))
end

9591844